In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append('/path/to/EquiNet') # location of EquiNet repository

from equinet.train import make_predictions
from equinet.args import PredictArgs

In [ ]:
# set your inputs
smiles1 = "CC(C)CCl"
smiles2 = "CC(C)OC(C)C"
T = 288 # Temperature in K

In [ ]:
# If you want a temporary path location for the results to write to
import tempfile

temp_preds = tempfile.mktemp(suffix='.csv')
temp_features = tempfile.mktemp(suffix='.csv')
temp_test = tempfile.mktemp(suffix='.csv')

In [ ]:
# make input files

x1s = np.arange(0, 1.01, 0.01)
x2s = 1 - x1s

test_df = pd.DataFrame({
    'smiles1': [smiles1]*len(x1s),
    'smiles2': [smiles2]*len(x2s),
})
features_df = pd.DataFrame({
    'x1': x1s,
    'x2': x2s,
    'T': [T]*len(x1s),
    'log10P1sat': ['nan']*len(x1s),
    'log10P2sat': ['nan']*len(x1s),
})

print(test_df)
print(features_df)

In [ ]:
test_df.to_csv(temp_test, index=False)
features_df.to_csv(temp_features, index=False)

In [ ]:
arguments = [
    '--test_path', temp_test,
    '--features_path', temp_features,
    '--preds_path', temp_preds,
    '--checkpoint_dir', '.',
    '--number_of_molecules', '2',
    '--num_workers', '0',
]

args = PredictArgs().parse_args(arguments)

In [ ]:
model_objects = load_model(args=args)

In [ ]:
preds = make_predictions(args=args, model_objects=model_objects)

In [ ]:
preds_df = pd.DataFrame(preds, columns=['y1', 'y2', 'log10P', 'lngamma1', 'lngamma2', 'log10P1sat', 'log10P2sat'])
preds_df

In [ ]:
plt.plot(x1s, 10**preds_df['log10P'],label="liquid", color='blue')
plt.plot(preds_df["y1"], 10**preds_df['log10P'], label="vapor", color='red')
plt.legend()
plt.xlim(0, 1)
plt.ylabel('Pressure, P (Pa)')
plt.xlabel('Mole Fraction of Component 1, x1 and y1')
plt.title(f'Mixture of SMILES {smiles1} (1) and \nSMILES {smiles2} (2) at T = {T} K')